In [6]:
import numpy as np
import csv
from memory_profiler import memory_usage
import timeit
import random

def sample_henyey_greenstein(g, generator):
    # p generator 0 to 1
    P = generator.uniform(0.0, 1.0)
    s = 2 * P - 1

    if g == 0:
        # isotropic scattering
        cos_theta = s
    else:
        # inverse cdf sampling
        cos_theta = (1 + g**2 - ((1 - g**2) / (1 + g * s))**2) / (2 * g)

    theta = np.arccos(cos_theta)
    # randomly choose the sign for the angle
    sign = generator.choice([-1, 1])
    phi = theta * sign

    return phi

def update_photon_angle(current_angle, g, generator):

    deflection = sample_henyey_greenstein(g, generator)
    

    new_angle = current_angle + deflection
    new_angle = new_angle % (2 * np.pi) 

    if new_angle < 0:
        new_angle += 2 * np.pi

    return new_angle

def photon_emitter(angle, v, sca_length, abs_length, init_coords, sensor_centers, sensor_r, g, generator):
    total_time = 0
    init_x, init_y = init_coords[0], init_coords[1]
    traj_x, traj_y = [init_x], [init_y]
    dist_traveled = 0
    while True:
        samp_sca = np.random.exponential(sca_length)
        samp_abs = np.random.exponential(abs_length)
        samp_dist = min(samp_sca, samp_abs)  # Distance until event

        dx = samp_dist * np.cos(angle)
        dy = samp_dist * np.sin(angle)
        prev_x, prev_y = traj_x[-1], traj_y[-1]
        new_x = prev_x + dx
        new_y = prev_y + dy
        traj_x, traj_y, update_time, index = update_traj(traj_x, traj_y, prev_x, prev_y, new_x, new_y, angle, v, sensor_centers, sensor_r)
        total_time += update_time
        dist_traveled += np.sqrt((new_x - prev_x) ** 2 + (new_y - prev_y) ** 2)
        if index >= -1:  # Meaning intersect or outside wall
            return traj_x, traj_y, total_time, dist_traveled, index
        if samp_dist == samp_abs:  # Absorption
            return traj_x, traj_y, total_time, dist_traveled, index
        else:
            angle = update_photon_angle(angle, g, generator)

def update_traj(traj_x, traj_y, init_x, init_y, final_x, final_y, angle, v, sensor_centers, sensor_r):
    dis = np.sqrt((final_x - init_x) ** 2 + (final_y - init_y) ** 2)
    speed = v
    time = dis / speed

    intersect, index, inter_coords = check_intersection(init_x, init_y, final_x, final_y, sensor_centers, sensor_r)
    if intersect:
        traj_x.append(inter_coords[0])
        traj_y.append(inter_coords[1])
        return traj_x, traj_y, time, index

    traj_x.append(final_x)
    traj_y.append(final_y)
    return traj_x, traj_y, time, -2

def check_intersection(prev_x, prev_y, curr_x, curr_y, sensor_centers, sensor_r):
    coords, index = check_sensors(prev_x, prev_y, curr_x, curr_y, sensor_centers, sensor_r)
    if index >= 0:
        return True, index, coords
    elif (curr_x <= 0 or curr_x >= 25 or curr_y <= 0 or curr_y >= 33):  # Outside walls
        curr_x, curr_y = get_wall_coords(prev_x, prev_y, curr_x, curr_y)
        return True, -1, (curr_x, curr_y)

    return False, -2, (0, 0)


def get_wall_coords(prev_x, prev_y, curr_x, curr_y):
    wall_boundaries = {'left': 0, 'right': 25, 'bottom': 0, 'top': 33}
    intersects = []

    if curr_x < wall_boundaries['left']:
        t = (wall_boundaries['left'] - prev_x) / (curr_x - prev_x)
        y_intersect = prev_y + t * (curr_y - prev_y)
        if 0 <= t <= 1 and wall_boundaries['bottom'] <= y_intersect <= wall_boundaries['top']:
            intersects.append((wall_boundaries['left'], y_intersect))

    if curr_x > wall_boundaries['right']:
        t = (wall_boundaries['right'] - prev_x) / (curr_x - prev_x)
        y_intersect = prev_y + t * (curr_y - prev_y)
        if 0 <= t <= 1 and wall_boundaries['bottom'] <= y_intersect <= wall_boundaries['top']:
            intersects.append((wall_boundaries['right'], y_intersect))

    if curr_y < wall_boundaries['bottom']:
        t = (wall_boundaries['bottom'] - prev_y) / (curr_y - prev_y)
        x_intersect = prev_x + t * (curr_x - prev_x)
        if 0 <= t <= 1 and wall_boundaries['left'] <= x_intersect <= wall_boundaries['right']:
            intersects.append((x_intersect, wall_boundaries['bottom']))

    if curr_y > wall_boundaries['top']:
        t = (wall_boundaries['top'] - prev_y) / (curr_y - prev_y)
        x_intersect = prev_x + t * (curr_x - prev_x)
        if 0 <= t <= 1 and wall_boundaries['left'] <= x_intersect <= wall_boundaries['right']:
            intersects.append((x_intersect, wall_boundaries['top']))

    return intersects[0]  # Return the first valid intersection



def check_sensors(prev_x, prev_y, curr_x, curr_y, sensor_centers, sensor_r):
    for i, (x_cent, y_cent) in enumerate(sensor_centers):
        A = prev_x - x_cent
        B = curr_x - prev_x
        C = prev_y - y_cent
        D = curr_y - prev_y

        a = B ** 2 + D ** 2
        b = 2 * (A * B + C * D)
        c = A ** 2 + C ** 2 - sensor_r ** 2

        discriminant = b ** 2 - 4 * a * c

        if discriminant < 0:
            continue 

        sqrt_discriminant = np.sqrt(discriminant)

        t1 = (-b - sqrt_discriminant) / (2 * a)
        t2 = (-b + sqrt_discriminant) / (2 * a)

        if (0 <= t1 <= 1) or (0 <= t2 <= 1):
            return (x_cent, y_cent), i

    return False, -2

def run_simulation(n_trajs, detec_x, detec_y, sensor_d, n_sensors, g):
    c = 3e8
    n_ice = 1.31
    c_ice = c / n_ice

    sensor_r = sensor_d / 2
    emitter_coords = (12, 17)

    sensor_x = np.linspace(sensor_d, detec_x - sensor_d, int(np.sqrt(n_sensors)))
    sensor_y = np.linspace(sensor_d, detec_y - sensor_d, int(np.sqrt(n_sensors)))

    X, Y = np.meshgrid(sensor_x, sensor_y)

    # Sensor centers array
    sensor_centers = np.vstack([X.ravel(), Y.ravel()]).T

    traj_times = np.zeros(n_trajs)
    traj_dist = np.zeros(n_trajs)
    sensor_hits = []
    sensor_hit_times = []
    all_trajs_x = []
    all_trajs_y = []

    no_sensor_dist = []
    no_sensor_times = []
    generator = np.random.default_rng()

    for i in range(n_trajs):
        traj_x, traj_y, final_time, dist_traveled, index = photon_emitter(np.random.uniform(0, 2 * np.pi), c_ice, 7, 11, init_coords=emitter_coords, sensor_centers=sensor_centers, sensor_r=sensor_r, g=g, generator=generator)
        traj_times[i] = final_time
        traj_dist[i] = dist_traveled
        all_trajs_x.append(traj_x)
        all_trajs_y.append(traj_y)

        if index >= 0:
            sensor_hits.append(index)
            sensor_hit_times.append(final_time)
        if index < 0:
            no_sensor_dist.append(dist_traveled)
            no_sensor_times.append(final_time)
    return traj_times, traj_dist, sensor_hits, sensor_hit_times, all_trajs_x, all_trajs_y, no_sensor_dist, no_sensor_times

def save_to_csv(filename, data):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)

def benchmark_and_save_results(detec_x, detec_y, sensor_d, n_sensors, g):
    n_photons_list = [300000]  # Use a smaller range for testing
    results = []

    for n_photons in n_photons_list:
        print(f"Running simulation for {n_photons} photons...")
        # mem_usage = memory_usage((run_simulation, (n_photons, detec_x, detec_y, sensor_d, n_sensors, g)))
        # time_usage = timeit.timeit(lambda: run_simulation(n_photons, detec_x, detec_y, sensor_d, n_sensors, g), number=1)
        
        traj_times, traj_dist, sensor_hits, sensor_hit_times, all_trajs_x, all_trajs_y, no_sensor_dist, no_sensor_times = run_simulation(n_photons, detec_x, detec_y, sensor_d, n_sensors, g)
        #print(no_sensor_dist)
        save_to_csv(f'all_trajs_x_large_{n_photons}.csv', all_trajs_x)
        save_to_csv(f'all_trajs_y_large_{n_photons}.csv', all_trajs_y)
        save_to_csv(f'no_sensor_dist_large_{n_photons}.csv', [no_sensor_dist])
        save_to_csv(f'no_sensor_times_large_{n_photons}.csv', [no_sensor_times])
        save_to_csv(f'traj_times_large_{n_photons}.csv', [traj_times])
        save_to_csv(f'traj_dist_large_{n_photons}.csv', [traj_dist])
        save_to_csv(f'sensor_hits_large_{n_photons}.csv', [sensor_hits])
        save_to_csv(f'sensor_hit_times_large_{n_photons}.csv', [sensor_hit_times])
        
        # results.append([n_photons, max(mem_usage), time_usage])
        # print(f"Finished simulation for {n_photons} photons: Memory Usage = {max(mem_usage)} MiB, Time Usage = {time_usage} seconds")

    # # Save benchmark results to CSV
    # with open('benchmark_results.csv', 'w', newline='') as csvfile:
    #     writer = csv.writer(csvfile)
    #     writer.writerow(['Number of Photons', 'Memory Usage (MiB)', 'Time Usage (seconds)'])
    #     writer.writerows(results)

    print("results saved to csvs")

# Parameters
detec_x = 80
detec_y = 60
sensor_d = 0.5
n_sensors = 144
g = 0.8  # Henyey-Greenstein asymmetry parameter

benchmark_and_save_results(detec_x, detec_y, sensor_d, n_sensors, g)


Running simulation for 300000 photons...
results saved to csvs


In [9]:
import numpy as np
import csv
import timeit

def sample_henyey_greenstein(g, generator):
    rand = generator.uniform(0.0, 1.0)

    if g == 0:
        cos_theta = 2 * rand - 1
    else:
        sqr_term = (1 - g * g) / (1 - g + 2 * g * rand)
        cos_theta = (1 + g * g - sqr_term * sqr_term) / (2 * g)

    theta = np.arccos(cos_theta)

    sign = generator.choice([-1, 1])
    phi = theta * sign

    return phi

def update_photon_angle(current_angle, g, generator):
    deflection = sample_henyey_greenstein(g, generator)
    new_angle = current_angle + deflection
    new_angle = new_angle % (2 * np.pi)

    if new_angle < 0:
        new_angle += 2 * np.pi

    return new_angle

def photon_emitter(angle, v, sca_length, abs_length, init_coords, sensor_centers, sensor_r, g, generator, detec_x, detec_y):
    total_time = 0
    init_x, init_y = init_coords[0], init_coords[1]
    traj_x, traj_y = [init_x], [init_y]
    dist_traveled = 0
    while True:
        samp_sca = np.random.exponential(sca_length)
        samp_abs = np.random.exponential(abs_length)
        samp_dist = min(samp_sca, samp_abs)  # Distance until event

        dx = samp_dist * np.cos(angle)
        dy = samp_dist * np.sin(angle)
        prev_x, prev_y = traj_x[-1], traj_y[-1]
        new_x = prev_x + dx
        new_y = prev_y + dy
        traj_x, traj_y, update_time, index = update_traj(traj_x, traj_y, prev_x, prev_y, new_x, new_y, angle, v, sensor_centers, sensor_r, detec_x, detec_y)
        total_time += update_time
        dist_traveled += np.sqrt((new_x - prev_x) ** 2 + (new_y - prev_y) ** 2)
        if index >= -1:  # Meaning intersect or outside wall
            return traj_x, traj_y, total_time, dist_traveled, index
        if samp_dist == samp_abs:  # Absorption
            return traj_x, traj_y, total_time, dist_traveled, index
        else:
            angle = update_photon_angle(angle, g, generator)

def update_traj(traj_x, traj_y, init_x, init_y, final_x, final_y, angle, v, sensor_centers, sensor_r, detec_x, detec_y):
    dis = np.sqrt((final_x - init_x) ** 2 + (final_y - init_y) ** 2)
    speed = v
    time = dis / speed

    intersect, index, inter_coords = check_intersection(init_x, init_y, final_x, final_y, sensor_centers, sensor_r, detec_x, detec_y)
    if intersect:
        traj_x.append(inter_coords[0])
        traj_y.append(inter_coords[1])
        return traj_x, traj_y, time, index

    traj_x.append(final_x)
    traj_y.append(final_y)
    return traj_x, traj_y, time, -2


def check_intersection(prev_x, prev_y, curr_x, curr_y, sensor_centers, sensor_r, detec_x, detec_y):
    coords, index = check_sensors(prev_x, prev_y, curr_x, curr_y, sensor_centers, sensor_r)
    if index >= 0:
        return True, index, coords
    elif (curr_x <= 0 or curr_x >= detec_x or curr_y <= 0 or curr_y >= detec_y):  # Outside walls
        curr_x, curr_y = get_wall_coords(prev_x, prev_y, curr_x, curr_y, detec_x, detec_y)
        return True, -1, (curr_x, curr_y)

    return False, -2, (0, 0)

def get_wall_coords(prev_x, prev_y, curr_x, curr_y, detec_x, detec_y):
    wall_boundaries = {'left': 0, 'right': detec_x, 'bottom': 0, 'top': detec_y}
    intersects = []

    if curr_x < wall_boundaries['left']:
        t = (wall_boundaries['left'] - prev_x) / (curr_x - prev_x)
        y_intersect = prev_y + t * (curr_y - prev_y)
        if 0 <= t <= 1 and wall_boundaries['bottom'] <= y_intersect <= wall_boundaries['top']:
            intersects.append((wall_boundaries['left'], y_intersect))

    if curr_x > wall_boundaries['right']:
        t = (wall_boundaries['right'] - prev_x) / (curr_x - prev_x)
        y_intersect = prev_y + t * (curr_y - prev_y)
        if 0 <= t <= 1 and wall_boundaries['bottom'] <= y_intersect <= wall_boundaries['top']:
            intersects.append((wall_boundaries['right'], y_intersect))

    if curr_y < wall_boundaries['bottom']:
        t = (wall_boundaries['bottom'] - prev_y) / (curr_y - prev_y)
        x_intersect = prev_x + t * (curr_x - prev_x)
        if 0 <= t <= 1 and wall_boundaries['left'] <= x_intersect <= wall_boundaries['right']:
            intersects.append((x_intersect, wall_boundaries['bottom']))

    if curr_y > wall_boundaries['top']:
        t = (wall_boundaries['top'] - prev_y) / (curr_y - prev_y)
        x_intersect = prev_x + t * (curr_x - prev_x)
        if 0 <= t <= 1 and wall_boundaries['left'] <= x_intersect <= wall_boundaries['right']:
            intersects.append((x_intersect, wall_boundaries['top']))

    if intersects:
        return intersects[0]
    else:
        return curr_x, curr_y


def check_sensors(prev_x, prev_y, curr_x, curr_y, sensor_centers, sensor_r):
    for i, (x_cent, y_cent) in enumerate(sensor_centers):
        A = prev_x - x_cent
        B = curr_x - prev_x
        C = prev_y - y_cent
        D = curr_y - prev_y

        a = B ** 2 + D ** 2
        b = 2 * (A * B + C * D)
        c = A ** 2 + C ** 2 - sensor_r ** 2

        discriminant = b ** 2 - 4 * a * c

        if discriminant < 0:
            continue 

        sqrt_discriminant = np.sqrt(discriminant)

        t1 = (-b - sqrt_discriminant) / (2 * a)
        t2 = (-b + sqrt_discriminant) / (2 * a)

        if (0 <= t1 <= 1) or (0 <= t2 <= 1):
            return (x_cent, y_cent), i

    return False, -2
def run_simulation(n_trajs, detec_x, detec_y, sensor_d, n_sensors, g, emitter_coords):
    c = 3e8
    n_ice = 1.31
    c_ice = c / n_ice

    sensor_r = sensor_d / 2

    sensor_x = np.linspace(sensor_d, detec_x - sensor_d, int(np.sqrt(n_sensors)))
    sensor_y = np.linspace(sensor_d, detec_y - sensor_d, int(np.sqrt(n_sensors)))

    X, Y = np.meshgrid(sensor_x, sensor_y)

    # Sensor centers array
    sensor_centers = np.vstack([X.ravel(), Y.ravel()]).T

    traj_times = np.zeros(n_trajs)
    traj_dist = np.zeros(n_trajs)
    sensor_hits = []
    sensor_hit_times = []
    all_trajs_x = []
    all_trajs_y = []

    no_sensor_dist = []
    no_sensor_times = []

    sensor_hit_indices = []

    generator = np.random.default_rng()

    for i in range(n_trajs):
        traj_x, traj_y, final_time, dist_traveled, index = photon_emitter(np.random.uniform(0, 2 * np.pi), c_ice, 7, 11, init_coords=emitter_coords, sensor_centers=sensor_centers, sensor_r=sensor_r, g=g, generator=generator, detec_x=detec_x, detec_y=detec_y)
        traj_times[i] = final_time
        traj_dist[i] = dist_traveled
        all_trajs_x.append(traj_x)
        all_trajs_y.append(traj_y)

        if index >= 0:
            sensor_hits.append(index)
            sensor_hit_times.append(final_time)
            sensor_hit_indices.append(i)
        if index < 0:
            no_sensor_dist.append(dist_traveled)
            no_sensor_times.append(final_time)

    return traj_times, traj_dist, sensor_hits, sensor_hit_times, all_trajs_x, all_trajs_y, no_sensor_dist, no_sensor_times, sensor_hit_indices

def save_to_csv(filename, data):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)

def benchmark_and_save_results(detec_x, detec_y, sensor_d, n_sensors, g, emitter_coords):
    n_photons_list = [300000]  # Use a smaller range for testing
    results = []

    for n_photons in n_photons_list:
        print(f"Running simulation for {n_photons} photons...")
        traj_times, traj_dist, sensor_hits, sensor_hit_times, all_trajs_x, all_trajs_y, no_sensor_dist, no_sensor_times, sensor_hit_indices = run_simulation(n_photons, detec_x, detec_y, sensor_d, n_sensors, g, emitter_coords)

        # Extract sensor trajectories and distances based on sensor_hit_indices
        sensor_trajs_x = [all_trajs_x[i] for i in sensor_hit_indices]
        sensor_trajs_y = [all_trajs_y[i] for i in sensor_hit_indices]
        sensor_dist = [traj_dist[i] for i in sensor_hit_indices]

        save_to_csv(f'all_trajs_x_large_{n_photons}.csv', all_trajs_x)
        save_to_csv(f'all_trajs_y_large_{n_photons}.csv', all_trajs_y)
        save_to_csv(f'no_sensor_dist_large_{n_photons}.csv', [[d] for d in no_sensor_dist])  # Wrap each value in a list
        save_to_csv(f'no_sensor_times_large_{n_photons}.csv', [[t] for t in no_sensor_times])  # Wrap each value in a list
        save_to_csv(f'traj_times_large_{n_photons}.csv', [[t] for t in traj_times])  # Wrap each value in a list
        save_to_csv(f'traj_dist_large_{n_photons}.csv', [[d] for d in traj_dist])  # Wrap each value in a list
        save_to_csv(f'sensor_hits_large_{n_photons}.csv', [[s] for s in sensor_hits])  # Wrap each value in a list
        save_to_csv(f'sensor_hit_times_large_{n_photons}.csv', [[t] for t in sensor_hit_times])  # Wrap each value in a list
        save_to_csv(f'sensor_dist_large_{n_photons}.csv', [[d] for d in sensor_dist])  # Wrap each value in a list
        save_to_csv(f'sensor_trajs_x_large_{n_photons}.csv', sensor_trajs_x)
        save_to_csv(f'sensor_trajs_y_large_{n_photons}.csv', sensor_trajs_y)


    #    save_to_csv(f'all_trajs_x_{n_photons}.csv', all_trajs_x)
    #     save_to_csv(f'all_trajs_y_{n_photons}.csv', all_trajs_y)
    #     save_to_csv(f'no_sensor_dist_{n_photons}.csv', [[d] for d in no_sensor_dist])  # Wrap each value in a list
    #     save_to_csv(f'no_sensor_times_{n_photons}.csv', [[t] for t in no_sensor_times])  # Wrap each value in a list
    #     save_to_csv(f'traj_times_{n_photons}.csv', [[t] for t in traj_times])  # Wrap each value in a list
    #     save_to_csv(f'traj_dist_{n_photons}.csv', [[d] for d in traj_dist])  # Wrap each value in a list
    #     save_to_csv(f'sensor_hits_{n_photons}.csv', [[s] for s in sensor_hits])  # Wrap each value in a list
    #     save_to_csv(f'sensor_hit_times_{n_photons}.csv', [[t] for t in sensor_hit_times])  # Wrap each value in a list
    #     save_to_csv(f'sensor_dist_{n_photons}.csv', [[d] for d in sensor_dist])  # Wrap each value in a list
    #     save_to_csv(f'sensor_trajs_x_{n_photons}.csv', sensor_trajs_x)
    #     save_to_csv(f'sensor_trajs_y_{n_photons}.csv', sensor_trajs_y)


        results.append([n_photons])

    # Save benchmark results to CSV
    # with open('benchmark_results.csv', 'w', newline='') as csvfile:
    #     writer = csv.writer(csvfile)
    #     writer.writerow(['Number of Photons'])
    #     writer.writerows(results)

    print("Benchmark results saved to 'benchmark_results.csv'")

# Parameters
detec_x = 80
detec_y = 60
sensor_d = .5
n_sensors = 100
g = 0.8  # Henyey-Greenstein asymmetry parameter
emitter_coords = (30, 30)
#$emitter_coords = (80, 60)
benchmark_and_save_results(detec_x, detec_y, sensor_d, n_sensors, g, emitter_coords)


Running simulation for 300000 photons...
Benchmark results saved to 'benchmark_results.csv'


In [24]:
no_sensor_dist

NameError: name 'no_sensor_dist' is not defined

In [11]:
detec_x = 25
detec_y = 33
sensor_d = 0.15
sensor_r = sensor_d/2
n_sensors = 100
c = 3e8
n_ice = 1.31
c_ice = c/n_ice

emitter_coords = (12,17)


benchmark_and_save_results(detec_x, detec_y, sensor_d, n_sensors)

Running simulation for 100000 photons...
Finished simulation for 100000 photons: Memory Usage = 132.84375 MiB, Time Usage = 116.66478450002614 seconds
Benchmark results saved to 'benchmark_results.csv'


In [48]:
import sys
print(sys.executable)


C:\Users\jackp\AppData\Local\Programs\Python\Python312\python.exe
